In [1]:
from pathlib import Path
import dask.dataframe as dd
import numpy as np
import datetime

In [2]:
# loading metadata file
filePath = 'path'
df = dd.read_csv(filePath + 'a.csv').drop_duplicates()

# Pre-processing to filter out inconsistent data
df = df[df['Number of Occupants'] < df['Number of Occupants'].quantile(.999)]
df = df[df['Number of Floors'] < df['Number of Floors'].quantile(.999)]
df = df[df['Floor Area [ft2]'] < df['Floor Area [ft2]'].quantile(.999)]
df = df[df['Number of Remote Sensors'] < df['Number of Remote Sensors'].quantile(.999)]

In [3]:

lista = []
# List all files in directory using pathlib
basepath = Path("e/")
files_in_basepath = (entry for entry in basepath.iterdir() if entry.is_file())
for item in files_in_basepath:
    lista.append(item.name)

In [4]:
lista_FileName = [s.replace('.csv', '') for s in lista]  # to have only the name of the file
df_in_2019 = df[df.Identifier.isin(lista_FileName)]

In [5]:
df1 = df_in_2019[(df_in_2019['Number of Occupants'] == 1 ) & (df_in_2019['Number of Remote Sensors'] >= 3)]

(946, 22)
(3939, 22)
(2506, 22)
(2879, 22)


In [6]:
use_cols = [cols]
mainPath ='path'

In [7]:
# autumn
start_date= datetime.datetime.strptime('2019-09-21','%Y-%m-%d')
end_date = datetime.datetime.strptime('2019-12-21','%Y-%m-%d')

In [8]:
##### DF1

# Take all Identifiers of df, to after open them
a=df1.loc[:,'Identifier'].compute().tolist()
lista=[]
for i in a:
    lista.append(mainPath+i+'.csv')
# Read all files 
ff=dd.read_csv(lista, usecols = use_cols, parse_dates=['DateTime'], include_path_column=True)
ff['path']=ff['path'].str.replace('F:/2019_unzipped_complete/', '')
ff['path']=ff['path'].str.replace('.csv', '')

# Manage of columns
ff['activation'] = ff.loc[:, ff.columns.str.contains('_Motion')].sum(axis=1) # sumo cantidad de rs en 1
ff['dayofweek']=ff['DateTime'].dt.day_name()
ff['Time'] = ff['DateTime'].dt.time
ff['Month'] = ff['DateTime'].dt.month

ff_columns = ['path', 'activation', 'dayofweek', 'Time', 'Month','DateTime']
result = dd.merge(df1[['Identifier','Number of Remote Sensors']], ff[ff_columns], left_on='Identifier', right_on='path').drop(['path'],axis=1)
result['mean_activation']=result['activation']/(result['Number of Remote Sensors']+1) # +1 counting sensor of thermostat
result=result[(result['DateTime'] >= start_date) & (result['DateTime']< end_date)]

aa=result[['dayofweek','Time','mean_activation']].groupby(['Time','dayofweek']).mean()
pp=dd.from_array(aa.to_records()) 

bb = pp.categorize(columns='Time')
table = dd.pivot_table(bb, values='mean_activation', index='dayofweek', columns='Time')

t=dd.from_array(table.to_records())
(t.compute()).to_csv('.csv')